# Convert IPUMS data into something Tableau can swallow

In [1]:
#imports
import os
import pandas as pd
import numpy as np

In [2]:
#get data
mydir = os.getcwd()
fulldata = pd.read_csv(os.path.join(mydir,'hhdata.csv'))

In [3]:
#initialize zeros array
numyears = fulldata.YEAR.unique()
counter = 0
for a in range(len(numyears)):
    somedata = fulldata.loc[fulldata["YEAR"] == numyears[a]]
    nowcounter = somedata.nunique()
    counter = counter + nowcounter[2]
output = np.zeros([counter,6])

In [4]:
currSerial = 0
currYear = 0
grpAge = np.zeros(30)
grpEmp = np.zeros(30)
j = 0
init=True
working = True
for i in range(len(fulldata)):
    if fulldata["YEAR"][i] > currYear:
        currSerial = 0
        currYear = fulldata["YEAR"][i]
    if fulldata["SERIAL"][i] > currSerial:
        if init:
            k=0
            currSerial = fulldata["SERIAL"][i]
            grpAge = np.zeros(30)
            grpEmp = np.zeros(30)
            grpAge[k] = fulldata["AGE"][i]
            grpEmp[k] = fulldata["EMPSTAT"][i]
            k+=1
            init = False
        else:
            #test if household has child under 18
            if np.min(grpAge[np.nonzero(grpAge)])<18:
                U18 = True
            else:
                U18 = False
            
            #test for working people in household
            grpEmpTest = grpEmp - 1
            numWorking = len(grpEmpTest) - np.count_nonzero(grpEmpTest)
            if numWorking>0:
                working = True
            else:
                working = False
            
            #take down values
            output[j][0] = fulldata["YEAR"][i-1]
            output[j][1] = fulldata["HHWT"][i-1]
            output[j][2] = fulldata["STATEFIP"][i-1]
            output[j][3] = fulldata["POVERTY"][i-1]
            output[j][4] = U18
            output[j][5] = working
            j+=1
            
            #reset counter
            k=0
            currSerial = fulldata["SERIAL"][i]
            grpAge = np.zeros(30)
            grpEmp = np.zeros(30)
            grpAge[k] = fulldata["AGE"][i]
            grpEmp[k] = fulldata["EMPSTAT"][i]
            k+=1
    else:
        grpAge[k] = fulldata["AGE"][i]
        grpEmp[k] = fulldata["EMPSTAT"][i]
        k+=1
    if i==len(fulldata)-1:
        #test if household has child under 18
        if np.min(grpAge[np.nonzero(grpAge)])<18:
            U18 = True
        else:
            U18 = False
            
        #test for working people in household
        grpEmpTest = grpEmp - 1
        numWorking = len(grpEmpTest) - np.count_nonzero(grpEmpTest)
        if numWorking>0:
            working = True
        else:
            working = False
            
        #take down values
        output[j][0] = fulldata["YEAR"][i]
        output[j][1] = fulldata["HHWT"][i]
        output[j][2] = fulldata["STATEFIP"][i]
        output[j][3] = fulldata["POVERTY"][i]
        output[j][4] = U18
        output[j][5] = working

In [5]:
pd.DataFrame(output)

0      1     2      3    4    5
0        2007.0   88.0   1.0  189.0  1.0  1.0
1        2007.0   89.0   1.0  501.0  1.0  1.0
2        2007.0   62.0   1.0  501.0  1.0  1.0
3        2007.0   24.0   1.0  501.0  0.0  1.0
4        2007.0  249.0   1.0   10.0  1.0  1.0
5        2007.0   64.0   1.0  501.0  1.0  1.0
6        2007.0   56.0   1.0  223.0  1.0  1.0
7        2007.0   27.0   1.0  423.0  0.0  1.0
8        2007.0   81.0   1.0  205.0  1.0  1.0
9        2007.0   21.0   1.0  220.0  0.0  1.0
10       2007.0   75.0   1.0  247.0  1.0  1.0
11       2007.0  193.0   1.0  501.0  0.0  1.0
12       2007.0   27.0   1.0  212.0  1.0  1.0
13       2007.0  133.0   1.0  109.0  1.0  0.0
14       2007.0   69.0   1.0  501.0  0.0  1.0
15       2007.0   86.0   1.0    7.0  1.0  0.0
16       2007.0   52.0   1.0  470.0  0.0  1.0
17       2007.0   64.0   1.0  388.0  0.0  1.0
18       2007.0  121.0   1.0  501.0  1.0  1.0
19       2007.0  106.0   1.0  378.0  0.0  1.0
20       2007.0   16.0   1.0  283.0  0.0  1.0
21       2007.0   63.0   1.0  369.0  0.0  1.0
22       2007.0   83.0   1.0  501.0  0.0  1.0
23       2007.0   19.0   1.0  501.0  0.0  1.0
24       2007.0   67.0   1.0  221.0  1.0  1.0
25       2007.0  122.0   1.0  139.0  0.0  1.0
26       2007.0   86.0   1.0  333.0  1.0  1.0
27       2007.0   61.0   1.0  501.0  0.0  1.0
28       2007.0   79.0   1.0  159.0  1.0  1.0
29       2007.0  110.0   1.0  469.0  0.0  1.0
...         ...    ...   ...    ...  ...  ...
9240020  2016.0  101.0  56.0  265.0  0.0  1.0
9240021  2016.0  106.0  56.0  486.0  0.0  1.0
9240022  2016.0   70.0  56.0  290.0  0.0  1.0
9240023  2016.0   72.0  56.0  485.0  0.0  1.0
9240024  2016.0  421.0  56.0  501.0  0.0  1.0
9240025  2016.0   95.0  56.0  501.0  0.0  1.0
9240026  2016.0  103.0  56.0  236.0  0.0  0.0
9240027  2016.0  204.0  56.0  501.0  0.0  1.0
9240028  2016.0   27.0  56.0  224.0  1.0  1.0
9240029  2016.0   66.0  56.0  170.0  1.0  1.0
9240030  2016.0  234.0  56.0  238.0  0.0  1.0
9240031  2016.0   69.0  56.0  269.0  1.0  1.0
9240032  2016.0   84.0  56.0  271.0  0.0  1.0
9240033  2016.0  115.0  56.0   52.0  0.0  1.0
9240034  2016.0  255.0  56.0  403.0  1.0  1.0
9240035  2016.0   68.0  56.0  501.0  0.0  1.0
9240036  2016.0   41.0  56.0  501.0  0.0  1.0
9240037  2016.0   93.0  56.0  501.0  0.0  1.0
9240038  2016.0  187.0  56.0  501.0  1.0  1.0
9240039  2016.0  150.0  56.0  321.0  0.0  1.0
9240040  2016.0  114.0  56.0  501.0  0.0  1.0
9240041  2016.0   61.0  56.0  501.0  0.0  1.0
9240042  2016.0  679.0  56.0    1.0  0.0  1.0
9240043  2016.0   94.0  56.0  501.0  1.0  1.0
9240044  2016.0  172.0  56.0   94.0  1.0  1.0
9240045  2016.0   24.0  56.0  490.0  1.0  1.0
9240046  2016.0   65.0  56.0  501.0  0.0  1.0
9240047  2016.0   73.0  56.0  179.0  0.0  1.0
9240048  2016.0   67.0  56.0  501.0  0.0  1.0
9240049  2016.0  314.0  56.0  132.0  0.0  1.0

[9240050 rows x 6 columns]

In [7]:
#output data
dfoutput = pd.DataFrame(output)
dfoutput.to_csv(os.path.join(mydir,'hhcleandata.csv'))